# Progetto di Social Computing A.A. 2025-2026 

Alice Blandino, Elisa Morettuzzo, Claudia Benedetti

### Analisi sulle pubblicazioni scientifiche degli afferenti del Dipartimento DMIF dell'Università degli Studi di Udine, sulla base dei dati scaricati da Scopus. 

In [ ]:
# pacchetti necessari (file requirements.txt per l'installazione)

import pandas as pd
import networkx as nx
from pyvis.network import Network
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.colors as mcolors
import numpy as np
import os
from IPython.display import display
import itertools
import plotly.graph_objects as go
import scipy
import re
from plotly.subplots import make_subplots

Caricamento del dataset e successiva pulizia:

In [35]:
# caricamento dataset
df=pd.read_csv('dataset.csv')
df.head()

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,"Lancia, G.; Dalpasso, M.","Lancia, Giuseppe G. (6701584197); Dalpasso, Ma...",6701584197; 6603897248,Speeding Up Floyd–Warshall’s Algorithm to Comp...,2025,Algorithms,18,9,560,NaN,...,NaN,NaN,NaN,English,Algorithms,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-105017372448
1,"Da Ros, F.; Di Gaspero, L.; Kletzander, L.; La...","Da Ros, Francesca (58134137500); Di Gaspero, L...",58134137500; 6505944235; 57194560339; 57188717...,Dynamic Temperature Control of Simulated Annea...,2025,NaN,NaN,NaN,NaN,184,...,9798400714658,NaN,NaN,English,GECCO - Proc. Genetic Evol. Comput. Conf.,Conference paper,Final,NaN,Scopus,2-s2.0-105013077196
2,"Bacchetti, E.; de Nardin, A.; Giannarini, G.; ...","Bacchetti, Emiliano (59995162800); de Nardin, ...",59995162800; 57259162600; 55883317500; 2448058...,A Deep Learning Model Integrating Clinical and...,2025,Cancers,17,13,2257,NaN,...,NaN,NaN,NaN,English,Cancers,Article,Final,All Open Access; Gold Open Access; Green Accep...,Scopus,2-s2.0-105010643815
3,"Ozkilinc, O.; Soler, M.A.; Giannozzi, P.; Apar...","Ozkilinc, Ozge (58138235700); Soler, Miguel A....",58138235700; 8718855800; 7004488307; 572046647...,The Single-Parameter Bragg–Williams Model for ...,2025,International Journal of Molecular Sciences,26,3,997,NaN,...,NaN,NaN,39940766.0,English,Int. J. Mol. Sci.,Article,Final,All Open Access; Gold Open Access; Green Accep...,Scopus,2-s2.0-85217742685
4,"Faletič, S.; Micheli, M.; Pospiec, G.","Faletič, Sergej (56595108900); Micheli, Marisa...",56595108900; 59664609000; 59665606800,Teaching and learning quantum entanglement: a ...,2025,Journal of Physics: Conference Series,2950,1,012025,NaN,...,9788394593742; 9781628905861,NaN,NaN,English,J. Phys. Conf. Ser.,Conference paper,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85219573912


In [36]:
# Analisi valori nulli e pari a 0 per colonna
num_articoli = df.shape[0]        # numero articoli
null_counts = df.isnull().sum()   # Conta valori nulli per colonna
zero_counts = (df == 0).sum()     # Conta valori pari a 0 per colonna
print(f"Numero di articoli: {num_articoli}")

# Riepilogo dei valori mancanti
missing_summary = pd.DataFrame({
    "null_values": null_counts,
    "zero_values": zero_counts
})
print(missing_summary)

# Soglia: un terzo dei valori 
limite = len(df)/3
cols_to_drop = []       # Lista delle colonne da eliminare
for col in df.columns:
    num_null = df[col].isna().sum()
    num_zero = 0
    if pd.api.types.is_numeric_dtype(df[col]):
        num_zero = (df[col] == 0).sum()
    unique_vals = df[col].dropna().unique()     # toglie le colonne con un solo valore ripetuto
    if len(unique_vals) <= 1:
        cols_to_drop.append(col)
    # Se la somma di nulli + zeri supera limite, togli la colonna
    if (num_null + num_zero) >= limite:
        cols_to_drop.append(col)
df_clean = df.drop(columns=cols_to_drop)
df_clean.to_csv("nuovo_dataset.csv", index=False)   # nuovo dataset
cols_to_drop

df=pd.read_csv("nuovo_dataset.csv")
num_colonne = df.shape[1]
print("numero di colonne: ", num_colonne)
df.head()   # 46 da mettere dopo


Numero di articoli: 143
                               null_values  zero_values
Authors                                  0            0
Author full names                        0            0
Author(s) ID                             0            0
Title                                    0            0
Year                                     0            0
Source title                            20            0
Volume                                  23            0
Issue                                   77            0
Art. No.                                95            0
Page start                              62            0
Page end                                63            0
Page count                               0          143
Cited by                                 0           27
DOI                                     12            0
Link                                     0            0
Affiliations                             0            0
Authors with affiliation

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Cited by,DOI,Link,...,Author Keywords,Index Keywords,References,Correspondence Address,Publisher,ISSN,Abbreviated Source Title,Document Type,Publication Stage,EID
0,"Lancia, G.; Dalpasso, M.","Lancia, Giuseppe G. (6701584197); Dalpasso, Ma...",6701584197; 6603897248,Speeding Up Floyd–Warshall’s Algorithm to Comp...,2025,Algorithms,18,0,10.3390/a18090560,https://www.scopus.com/inward/record.uri?eid=2...,...,all-pairs shortest paths; FastSet data structu...,Computational complexity; Graph algorithms; Al...,"Floyd, Robert W., Algorithm 97: Shortest path,...","G. Lancia; Department of Mathematics, Computer...",Multidisciplinary Digital Publishing Institute...,19994893,Algorithms,Article,Final,2-s2.0-105017372448
1,"Da Ros, F.; Di Gaspero, L.; Kletzander, L.; La...","Da Ros, Francesca (58134137500); Di Gaspero, L...",58134137500; 6505944235; 57194560339; 57188717...,Dynamic Temperature Control of Simulated Annea...,2025,NaN,NaN,0,10.1145/3712256.3726390,https://www.scopus.com/inward/record.uri?eid=2...,...,combinatorial optimization; empirical analysis...,Application programs; Artificial intelligence;...,"Adriaensen, Steven, Fair-share ILS: A simple s...",NaN,"Association for Computing Machinery, Inc",NaN,GECCO - Proc. Genetic Evol. Comput. Conf.,Conference paper,Final,2-s2.0-105013077196
2,"Bacchetti, E.; de Nardin, A.; Giannarini, G.; ...","Bacchetti, Emiliano (59995162800); de Nardin, ...",59995162800; 57259162600; 55883317500; 2448058...,A Deep Learning Model Integrating Clinical and...,2025,Cancers,17,0,10.3390/cancers17132257,https://www.scopus.com/inward/record.uri?eid=2...,...,artificial intelligence; biopsy; magnetic reso...,alpha adrenergic receptor blocking agent; pros...,"Cornford, Philip A., EAU-EANM-ESTRO-ESUR-ISUP-...","G. Giannarini; Urology Unit, University Hospit...",Multidisciplinary Digital Publishing Institute...,20726694,Cancers,Article,Final,2-s2.0-105010643815
3,"Ozkilinc, O.; Soler, M.A.; Giannozzi, P.; Apar...","Ozkilinc, Ozge (58138235700); Soler, Miguel A....",58138235700; 8718855800; 7004488307; 572046647...,The Single-Parameter Bragg–Williams Model for ...,2025,International Journal of Molecular Sciences,26,0,10.3390/ijms26030997,https://www.scopus.com/inward/record.uri?eid=2...,...,binary mixtures; Bragg–Williams; eutectic solv...,deep eutectic solvent; solvent; Article; Bragg...,"Abbott, Andrew P., Deep Eutectic Solvents form...",F. Fogolari; Dipartimento di Scienze Matematic...,Multidisciplinary Digital Publishing Institute...,14220067; 16616596,Int. J. Mol. Sci.,Article,Final,2-s2.0-85217742685
4,"Faletič, S.; Micheli, M.; Pospiec, G.","Faletič, Sergej (56595108900); Micheli, Marisa...",56595108900; 59664609000; 59665606800,Teaching and learning quantum entanglement: a ...,2025,Journal of Physics: Conference Series,2950,0,10.1088/1742-6596/2950/1/012025,https://www.scopus.com/inward/record.uri?eid=2...,...,NaN,Quantum optics; Empirical research; Fundamenta...,"J Phys Conf Ser, (2021); Qtedu Consortium 2021...","S. Faletič; University of Ljubljana, Faculty o...",Institute of Physics,17426588; 17426596,J. Phys. Conf. Ser.,Conference paper,Final,2-s2.0-85219573912


## Grafi

### Primo grafo: 
collaborazioni tra autori negli anni 2017-2025

In [43]:
# Preprocessing

df["authors_list"] = df["Authors"].str.split(";").apply(
    lambda x: [a.strip().replace(",", "") for a in x]
)
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")

# Creazione grafi per anno
grafi_per_anno = {}
for anno, df_anno in df.groupby("Year"):
    G = nx.Graph()
    for authors in df_anno["authors_list"].dropna():
        for a1, a2 in itertools.combinations(authors, 2):
            if G.has_edge(a1, a2):
                G[a1][a2]["weight"] += 1
            else:
                G.add_edge(a1, a2, weight=1)
    grafi_per_anno[anno] = G

# Layout globale per tutti i grafi (coerenza tra anni)
G_totale = nx.compose_all(grafi_per_anno.values())
pos = nx.spring_layout(G_totale, seed=42)


# Funzione di plotting dei nodi

def create_traces(G, pos):
    edge_x, edge_y = [], []
    for u, v in G.edges():
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        mode="lines",
        line=dict(width=1, color="lightgray"),
        hoverinfo="none",
        name="Collaborazioni"
    )

    node_x, node_y, hover_text, degrees = [], [], [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        deg = G.degree(node)
        degrees.append(deg)
        hover_text.append(f"{node}<br>Collaborazioni: {deg}")

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode="markers",
        hoverinfo="text",
        text=hover_text,
        marker=dict(
            size=10,
            color=degrees,
            colorscale="Blues",
            cmin=0,
            cmax=max(degrees),
            showscale=True,
            colorbar=dict(title="Collaborazioni"),
            line=dict(width=0.5, color="black")
        ),
        name="Autori"
    )

    return edge_trace, node_trace


# Creazione dei frame per l'animazione

frames = []
for anno in sorted(grafi_per_anno.keys()):
    edge_trace, node_trace = create_traces(grafi_per_anno[anno], pos)
    frames.append(go.Frame(
        data=[edge_trace, node_trace],
        name=str(anno),
        layout=go.Layout(title_text=f"Collaborazioni tra autori – {anno}")
    ))


# Figura finale

fig = go.Figure(
    data=frames[0].data,
    frames=frames,
    layout=go.Layout(
        title="Collaborazioni tra autori nel tempo",
        paper_bgcolor="white",
        plot_bgcolor="white",
        xaxis=dict(range=[-1.1, 1.1], showgrid=False, zeroline=False, visible=False),
        yaxis=dict(range=[-1.1, 1.1], showgrid=False, zeroline=False, visible=False),
        updatemenus=[{
            "type": "buttons",
            "showactive": False,
            "buttons":[{
                "label": "Play",
                "method": "animate",
                "args":[None, {"frame": {"duration": 2000, "redraw": True},
                               "transition": {"duration": 500},
                               "fromcurrent": True}]
            }]
        }],
        sliders=[{
            "steps":[{"method":"animate","args":[[f.name]],"label":f.name} for f in frames]
        }]
    )
)

fig.show()


### Secondo grafo:
collaborazioni degli autori con il nodo più centrale di ogni anno (2017-2025)

In [ ]:
import networkx as nx
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# NODO CENTRALE + VICINI EQUICENTRALI
central_node = {}
highlight_nodes = {}

for anno, G in grafi_per_anno.items():
    if len(G) == 0:
        continue

    central = max(G.degree, key=lambda x: x[1])[0]
    deg_c = G.degree(central)

    central_node[anno] = central
    highlight_nodes[anno] = [central] + [
        n for n in G.neighbors(central) if G.degree(n) == deg_c
    ]

# LAYOUT CON NODO CENTRALE FISSO + NORMALIZZAZIONE
def compute_layout(G, central):
    pos = nx.spring_layout(
        G,
        seed=42,
        pos={central: (0, 0)},
        fixed=[central],
        k=0.5,
        iterations=100
    )

    xs, ys = zip(*pos.values())
    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)

    for n, (x, y) in pos.items():
        pos[n] = (
            -1 + 2 * (x - min_x) / (max_x - min_x) if max_x > min_x else 0,
            -1 + 2 * (y - min_y) / (max_y - min_y) if max_y > min_y else 0
        )
    return pos


# FUNZIONE PLOTLY
def plot_graph(G, pos, highlight=None, legend=True):
    edge_x, edge_y = [], []
    for u, v in G.edges():
        edge_x += [pos[u][0], pos[v][0], None]
        edge_y += [pos[u][1], pos[v][1], None]

    edge = go.Scatter(
        x=edge_x, y=edge_y,
        mode="lines",
        line=dict(width=1, color="gray"),
        hoverinfo="none",
        showlegend=False
    )

    max_deg = max(dict(G.degree()).values())
    node_x, node_y, text, color, size = [], [], [], [], []

    for n in G.nodes():
        node_x.append(pos[n][0])
        node_y.append(pos[n][1])
        d = G.degree(n)
        text.append(f"{n}<br>Grado: {d}")

        if highlight and n in highlight:
            color.append("crimson")
            size.append(11)
        else:
            color.append("royalblue" if d < max_deg else "crimson")
            size.append(8)

    node = go.Scatter(
        x=node_x, y=node_y,
        mode="markers",
        text=text,
        hoverinfo="text",
        marker=dict(size=size, color=color, line=dict(width=0.5, color="black")),
        showlegend=legend
    )

    return edge, node


# SUBPLOT 2x5 (CENTRALE + VICINI)
anni = sorted(central_node)

fig = make_subplots(
    rows=2, cols=5,
    subplot_titles=[
        f"{a}<br>{'<br>'.join(', '.join(highlight_nodes[a][i:i+2]) for i in range(0, min(4,len(highlight_nodes[a])),2))}"
        for a in anni
    ],
    horizontal_spacing=0.03,
    vertical_spacing=0.08
)

for i, anno in enumerate(anni):
    central = central_node[anno]
    H = grafi_per_anno[anno].subgraph([central] + list(grafi_per_anno[anno].neighbors(central)))
    pos = compute_layout(H, central)

    edge, node = plot_graph(H, pos, highlight_nodes[anno], legend=False)

    r, c = (1, i + 1) if i < 5 else (2, i - 4)
    fig.add_traces([edge, node], rows=r, cols=c)

fig.update_layout(
    height=650, width=1400,
    paper_bgcolor="white",
    plot_bgcolor="white",
    font=dict(size=10),
    margin=dict(l=20, r=20, t=50, b=20)
)

for r in [1, 2]:
    for c in range(1, 6):
        fig.update_xaxes(visible=False, row=r, col=c)
        fig.update_yaxes(visible=False, row=r, col=c)

fig.show()


### Terzo grafo: 
collaborazioni con l'autore più centrale di tutti gli anni (2017-2025) 

In [65]:
import networkx as nx
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ============================================================
# GRAFO TOTALE E AUTORE CENTRALE
# ============================================================
G_totale = nx.compose_all(grafi_per_anno.values())
central_author = max(G_totale.degree, key=lambda x: x[1])[0]

# ============================================================
# GRAFI CUMULATIVI
# ============================================================
grafi_cumulativi = {}
G_cum = nx.Graph()

for anno in sorted(grafi_per_anno):
    G_cum = nx.compose(G_cum, grafi_per_anno[anno])
    grafi_cumulativi[anno] = G_cum.copy()

# ============================================================
# LAYOUT (centrale fisso + normalizzazione)
# ============================================================
def compute_layout(G, central):
    pos = nx.spring_layout(
        G,
        seed=42,
        pos={central: (0, 0)},
        fixed=[central],
        k=0.5,
        iterations=200
    )

    xs, ys = zip(*pos.values())
    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)

    for n, (x, y) in pos.items():
        pos[n] = (
            -1.1 + 2.2 * (x - min_x) / (max_x - min_x) if max_x > min_x else 0,
            -1.1 + 2.2 * (y - min_y) / (max_y - min_y) if max_y > min_y else 0
        )
    return pos

# ============================================================
# FUNZIONE PLOT
# ============================================================
def plot_graph(G, pos, central):
    edge_x, edge_y = [], []
    for u, v in G.edges():
        edge_x += [pos[u][0], pos[v][0], None]
        edge_y += [pos[u][1], pos[v][1], None]

    edge = go.Scatter(
        x=edge_x, y=edge_y,
        mode="lines",
        line=dict(width=1, color="gray"),
        hoverinfo="none",
        showlegend=False
    )

    node_x, node_y, text = [], [], []
    for n in G.nodes():
        if n == central:
            continue
        node_x.append(pos[n][0])
        node_y.append(pos[n][1])
        text.append(f"{n}<br>Collaborazioni: {G.degree(n)}")

    nodes = go.Scatter(
        x=node_x, y=node_y,
        mode="markers",
        hoverinfo="text",
        text=text,
        marker=dict(size=8, color="royalblue", line=dict(width=0.5, color="black")),
        showlegend=False
    )

    x_c, y_c = pos[central]
    central_node = go.Scatter(
        x=[x_c], y=[y_c],
        mode="markers",
        hoverinfo="text",
        text=[f"{central}<br>Collaborazioni: {G.degree(central)}"],
        marker=dict(size=15, color="crimson", line=dict(width=0.5, color="black")),
        showlegend=False
    )

    return edge, nodes, central_node

# ============================================================
# SUBPLOT STATICO (2 x N)
# ============================================================
anni = sorted(grafi_cumulativi)
cols = min(5, len(anni))
rows = (len(anni) + cols - 1) // cols

fig = make_subplots(
    rows=rows,
    cols=cols,
    subplot_titles=[
        f"{a}" for a in anni
    ],
    horizontal_spacing=0.03,
    vertical_spacing=0.08
)

for i, anno in enumerate(anni):
    G = grafi_cumulativi[anno]

    if central_author not in G:
        continue

    nodes_cc = nx.node_connected_component(G, central_author)
    G_sub = G.subgraph(nodes_cc).copy()
    pos = compute_layout(G_sub, central_author)

    edge, nodes, central = plot_graph(G_sub, pos, central_author)

    r = i // cols + 1
    c = i % cols + 1

    fig.add_traces([edge, nodes, central], rows=r, cols=c)

# ============================================================
# LAYOUT FINALE
# ============================================================
fig.update_layout(
    height=350 * rows,
    width=300 * cols,
    paper_bgcolor="white",
    plot_bgcolor="white",
    font=dict(size=11),
    title=f"Collaborazioni cumulative – Autore centrale: {central_author}",
    title_x=0.5,
    margin=dict(l=20, r=20, t=60, b=20)
)

for r in range(1, rows + 1):
    for c in range(1, cols + 1):
        fig.update_xaxes(visible=False, row=r, col=c)
        fig.update_yaxes(visible=False, row=r, col=c)

fig.show()


In [62]:
import pandas as pd
import networkx as nx

records = []

for anno, G in grafi_cumulativi.items():

    if central_author not in G:
        continue

    nodes_cc = nx.node_connected_component(G, central_author)
    G_sub = G.subgraph(nodes_cc).copy()

    deg = dict(G_sub.degree())
    n = len(G_sub)

    # ranking
    sorted_deg = sorted(deg.items(), key=lambda x: x[1], reverse=True)
    rank = {node: i+1 for i, (node, _) in enumerate(sorted_deg)}

    betw = nx.betweenness_centrality(G_sub)
    eig = nx.eigenvector_centrality(G_sub, max_iter=1000)

    records.append({
        "Anno": anno,
        "Grado": deg[central_author],
        "Grado normalizzato": round(deg[central_author] / (n-1), 3),
        "Rank grado": rank[central_author],
        "Betweenness": round(betw[central_author], 3),
        "Eigenvector": round(eig[central_author], 3),
        "Nodi totali": n
    })

df_centrality = pd.DataFrame(records)
df_centrality


,Anno,Grado,Grado normalizzato,Rank grado,Betweenness,Eigenvector,Nodi totali
0,2019,21,1.000,1,0.326,0.388,22
1,2020,96,1.000,1,0.441,0.163,97
2,2021,112,0.848,1,0.482,0.156,133
3,2022,112,0.732,1,0.422,0.156,154
4,2023,118,0.529,1,0.455,0.157,224
5,2024,118,0.490,1,0.447,0.157,242
6,2025,118,0.456,1,0.428,0.157,260


L’autore Gigli G.L. risulta stabilmente centrale lungo l’intero periodo osservato, mantenendo il primo posto per grado e valori elevati di betweenness ed eigenvector centrality in tutti gli anni analizzati 

In [ ]:
import plotly.express as px

fig = px.line(
    df_centrality,
    x="Anno",
    y=["Grado normalizzato", "Betweenness", "Eigenvector"],
    markers=True,
    title=f"Evoluzione della centralità di {central_author}"
)

fig.show()
